<a href="https://colab.research.google.com/github/yuchu1996/cy-web.github.io/blob/master/Copy_of_Copy_of_a3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A3: TensorBoard

## About

In this assignment, you will design and run experiments to evaluate the impact of a few common parameters (like the choice of activation function, optimizer, and weight initialization strategy) and visualize the results in TensorBoard.

The starter code below shows the mechanics of using TensorBoard in Colab. Unlike the previous assignments, a limited amount of starter code is provided.  3c is an extra credit question, it's optional (you can receive full credit on this assignment without submitting it).

## Questions

### 3a. 
**Implement ReLU and compare against a previous activation function**.

The year is 2010. It is not commonly known that ReLU is a useful alternative to activation functions like Sigmoid or Tanh (nor has ReLU been implemented in the library you're using). Create a DNN to classify MNIST, and provide your own implementation of ReLU (instead of using a built-in method). Design and run an experiment to compare ReLU against other methods, and use TensorBoard to display your results. What differences do you observe, and why?

### 3b. 

**Optimizer and initalizer and soup**.

Do optimizers like Momentum or Adam really make a difference? How about different weight initialize strategies (like random normal, or glorot uniform?) Design and run experiments to find out, and use TensorBoard to display your results. What differences do you observe, and why?

### 3c. Extra credit (optional)

**Demonstrate the vanishing gradient problem**. 

Implement an especially deep neural network and train it on a simple dataset like MNIST. Choose activation functions, initialization strategies, and an optimizer that are likely to cause this behavior. Produce histograms of activations and gradients at various layers during training. What do you see? Next, adjust the parameters above to correct this behavior. Visualize and compare the results.

## Submission instructions

Please submit your assignment on CourseWorks by uploading a zip file that includes:

* A Jupyter notebook, containing complete code to reproduce your experiments, and saved output showing your results.

* A README file (plaintext is fine). This should contain your written conclusions for each question. These can be brief (a couple paragraphs). Try to be specific in your answers (if ReLU outperfoms sigmoid, try to answer why).

* Plots / diagrams (.jpgs). Since it is not convenient to save TensorBoard diagrams directly in a Jupyter notebook, you can take screenshots of your plots and submit them along with your Jupyter notebook in a zip file on CourseWorks. Please name your diagrams appropriately, and refer to them names in your notebook.

If you are working in Colab, you can prepare your notebook for submission by ensuring that runs end-to-end, then saving and downloading it:

1. ```Runtime -> Restart and run all```
1. ```File -> Save```
1. ```File -> Download.ipynb```

## Starter code for TensorBoard

In [0]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [0]:
import tensorflow as tf
assert tf.__version__ >= "2.0"

In [0]:
%load_ext tensorboard 

**Caution**. The following cell will clear the logs directory. If you're running this on your local machine, be careful executing it.

In [0]:
!rm -rf ./tensorboard-logs/ # Clear any logs from previous runs

Import a dataset

In [0]:
mnist = tf.keras.datasets.mnist
(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train, x_test = x_train / 255.0, x_test / 255.0

## First style

Define a simple model

In [0]:
from tensorflow.keras.layers import Dense, Flatten

def create_model():
  model = tf.keras.models.Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(128, activation='softmax'),
  ])
  return model

Create a logs directory

In [0]:
import datetime 
import os
date = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
log_dir = os.path.join("./tensorboard-logs/", date)
print("Writing logs to", log_dir)

Writing logs to ./tensorboard-logs/20191029-003548


### Run an experiment

In [0]:
from tensorflow.keras.optimizers import SGD

model = create_model() 
opt = SGD(learning_rate=0.001, momentum=0.0, nesterov=False) 
model.compile(optimizer=opt,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

exp_dir = os.path.join(log_dir, "exp1")

tb_callback = tf.keras.callbacks.TensorBoard(log_dir=exp_dir)

model.fit(x_train,
          y_train,
          epochs=1, 
          validation_data=(x_test, y_test), 
          callbacks=[tb_callback])

Train on 60000 samples, validate on 10000 samples
60000/60000 [==============================] - 8s 134us/sample - loss: 2.5281 - accuracy: 0.5613 - val_loss: 1.4181 - val_accuracy: 0.7648


### Run a second experiment

In [0]:
model = create_model() 
opt = SGD(learning_rate=0.001, momentum=0.9, nesterov=True)
model.compile(optimizer=opt,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

exp_dir = os.path.join(log_dir, "exp2")

tb_callback = tf.keras.callbacks.TensorBoard(log_dir=exp_dir)

model.fit(x_train,
          y_train,
          epochs=10, 
          validation_data=(x_test, y_test), 
          callbacks=[tb_callback])

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 7s 111us/sample - loss: 0.9131 - accuracy: 0.8047 - val_loss: 0.4883 - val_accuracy: 0.8794
Epoch 2/10
60000/60000 [==============================] - 6s 105us/sample - loss: 0.4635 - accuracy: 0.8796 - val_loss: 0.4039 - val_accuracy: 0.8956
Epoch 3/10
60000/60000 [==============================] - 6s 106us/sample - loss: 0.4074 - accuracy: 0.8898 - val_loss: 0.3697 - val_accuracy: 0.9018
Epoch 4/10
60000/60000 [==============================] - 6s 106us/sample - loss: 0.3797 - accuracy: 0.8964 - val_loss: 0.3506 - val_accuracy: 0.9062
Epoch 5/10
60000/60000 [==============================] - 6s 105us/sample - loss: 0.3625 - accuracy: 0.9003 - val_loss: 0.3376 - val_accuracy: 0.9090
Epoch 6/10
60000/60000 [==============================] - 6s 106us/sample - loss: 0.3501 - accuracy: 0.9036 - val_loss: 0.3287 - val_accuracy: 0.9092
Epoch 7/10
60000/60000 [==========================

### Start TensorBoard and compare the results

In [0]:
%tensorboard --logdir "$log_dir"

## Second style
Using a Subclassed model and a GradientTape

Prepre the dataset

In [0]:
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test))
train_ds = train_ds.shuffle(60000).batch(32)
test_ds = test_ds.batch(32)

Define a simple model

In [0]:
class MyModel(tf.keras.Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.flatten = Flatten(input_shape=(28, 28))
    self.d1 = Dense(10, activation='softmax')

  def call(self, x):
    x = self.flatten(x)
    return self.d1(x)

model = MyModel()

In [0]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

In [0]:
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')
train_loss = tf.keras.metrics.Mean(name='train_loss')
test_loss = tf.keras.metrics.Mean(name='test_loss')

Training and testing routines

In [0]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    predictions = model(images)
    loss = loss_fn(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

@tf.function
def test_step(images, labels):
  predictions = model(images)
  t_loss = loss_fn(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

Prepare log writers (previously, these were handled by the callback)

In [0]:
date = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
log_dir = os.path.join("./tensorboard-logs/", date)
print("Writing logs to", log_dir)

train_writer = tf.summary.create_file_writer(os.path.join(log_dir, "train"))
test_writer = tf.summary.create_file_writer(os.path.join(log_dir, "test"))

Writing logs to ./tensorboard-logs/20191028-231400


Train and log summaries

In [0]:
EPOCHS = 2

for epoch in range(EPOCHS):
  
  for images, labels in train_ds:
    train_step(images, labels)
    
  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
  print(template.format(epoch+1,
                        train_loss.result(),
                        train_accuracy.result()*100,
                        test_loss.result(),
                        test_accuracy.result()*100))
  
  with train_writer.as_default():
    tf.summary.scalar('accuracy', train_accuracy.result(), step=epoch)
    tf.summary.scalar('loss', train_loss.result(), step=epoch)

    # ====
    # Demo: show how to use histogram summaries
    # Create and log some random data
    # Useful if you're attemping the extra credit question
    # ====
    data = tf.random.normal((32, 100))
    tf.summary.histogram('random', 
                         data,
                         step=epoch, 
                         description='Your description')
    
  with test_writer.as_default():
    tf.summary.scalar('accuracy', test_accuracy.result(), step=epoch)
    tf.summary.scalar('loss', test_loss.result(), step=epoch)
    
  # Reset the metrics for the next epoch
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

Epoch 1, Loss: 0.47127506136894226, Accuracy: 87.55500030517578, Test Loss: 0.31238049268722534, Test Accuracy: 91.39999389648438
Epoch 2, Loss: 0.30420851707458496, Accuracy: 91.5433349609375, Test Loss: 0.2801072299480438, Test Accuracy: 92.2699966430664


In [0]:
%tensorboard --logdir "$log_dir"

## 3a ReLU acticvation function

In [0]:
def my_relu(x):
  return tf.math.maximum(0.0,x)

In [0]:
from tensorflow.keras import Model

In [0]:
from tensorflow.keras.layers import Dense, Flatten
import datetime 
import os

def create_model(act):
  model = tf.keras.models.Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(128, activation=act),
    Dense(64, activation=act),
    Dense(10, activation='softmax'),
  ])
  return model

In [0]:
###logs
date = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
log_dir = os.path.join("./tensorboard-logs/", date)
print("Writing logs to", log_dir)

Writing logs to ./tensorboard-logs/20191029-004750


#### run a experiment

In [0]:
from tensorflow.keras.optimizers import SGD


model = create_model(my_relu) 
opt = SGD(learning_rate=0.001, momentum=0.0, nesterov=False) 
model.compile(optimizer=opt,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

exp_dir = os.path.join(log_dir, "relu")

tb_callback = tf.keras.callbacks.TensorBoard(log_dir=exp_dir)

model.fit(x_train,
          y_train,
          epochs=10, 
          validation_data=(x_test, y_test), 
          callbacks=[tb_callback])

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 8s 132us/sample - loss: 3.5603 - accuracy: 0.1522 - val_loss: 2.0766 - val_accuracy: 0.4711
Epoch 2/10
60000/60000 [==============================] - 7s 115us/sample - loss: 1.6804 - accuracy: 0.6077 - val_loss: 1.2764 - val_accuracy: 0.6976
Epoch 3/10
60000/60000 [==============================] - 7s 114us/sample - loss: 1.0507 - accuracy: 0.7389 - val_loss: 0.8501 - val_accuracy: 0.7777
Epoch 4/10
60000/60000 [==============================] - 7s 115us/sample - loss: 0.7660 - accuracy: 0.7926 - val_loss: 0.6618 - val_accuracy: 0.8217
Epoch 5/10
60000/60000 [==============================] - 7s 116us/sample - loss: 0.6281 - accuracy: 0.8253 - val_loss: 0.5621 - val_accuracy: 0.8421
Epoch 6/10
60000/60000 [==============================] - 7s 115us/sample - loss: 0.5470 - accuracy: 0.8475 - val_loss: 0.4972 - val_accuracy: 0.8634
Epoch 7/10
60000/60000 [==========================

#### run another experiment

In [0]:
model = create_model("tanh") 
opt = SGD(learning_rate=0.001, momentum=0.0, nesterov=False)
model.compile(optimizer=opt,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

exp_dir = os.path.join(log_dir, "tanh")

tb_callback = tf.keras.callbacks.TensorBoard(log_dir=exp_dir)

model.fit(x_train,
          y_train,
          epochs=10, 
          validation_data=(x_test, y_test), 
          callbacks=[tb_callback])

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 7s 122us/sample - loss: 3.5536 - accuracy: 0.3125 - val_loss: 2.4331 - val_accuracy: 0.5323
Epoch 2/10
60000/60000 [==============================] - 7s 114us/sample - loss: 2.0011 - accuracy: 0.5930 - val_loss: 1.6746 - val_accuracy: 0.6470
Epoch 3/10
60000/60000 [==============================] - 7s 113us/sample - loss: 1.4703 - accuracy: 0.6970 - val_loss: 1.2818 - val_accuracy: 0.7454
Epoch 4/10
60000/60000 [==============================] - 7s 114us/sample - loss: 1.1511 - accuracy: 0.7665 - val_loss: 1.0212 - val_accuracy: 0.7964
Epoch 5/10
60000/60000 [==============================] - 7s 116us/sample - loss: 0.9335 - accuracy: 0.8059 - val_loss: 0.8397 - val_accuracy: 0.8257
Epoch 6/10
60000/60000 [==============================] - 7s 114us/sample - loss: 0.7827 - accuracy: 0.8302 - val_loss: 0.7147 - val_accuracy: 0.8454
Epoch 7/10
60000/60000 [==========================

In [0]:
%tensorboard --logdir "$log_dir"

From the tensor board of relu and tanh activation function, we can see that in 10 epoches, model using relu as activation function tend to have higher accuracy and low loss than that using tanh. So I conclude that relu has a better performance in this case.

## 3b Optimizer and initalizer and soup

### for Momentum and Adam

In [0]:
class MyModel(Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.flatten = Flatten()
    self.d1 = Dense(10)

  def call(self, x):
    x = self.flatten(x)
    x = self.d1(x)
    return my_relu(x)
  
  model = MyModel

In [0]:
date = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
log_dir = os.path.join("./tensorboard-logs/", date)
print("Writing logs to", log_dir)

Writing logs to ./tensorboard-logs/20191029-005905


In [0]:
model = create_model("relu")
opt = SGD(learning_rate=0.001, momentum=0.0, nesterov=False)
model.compile(optimizer=opt,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

exp_dir = os.path.join(log_dir, "opt1")

tb_callback = tf.keras.callbacks.TensorBoard(log_dir=exp_dir)

model.fit(x_train,
          y_train,
          epochs=10, 
          validation_data=(x_test, y_test), 
          callbacks=[tb_callback])

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 7s 122us/sample - loss: 3.2595 - accuracy: 0.2337 - val_loss: 2.0334 - val_accuracy: 0.5338
Epoch 2/10
60000/60000 [==============================] - 7s 113us/sample - loss: 1.6893 - accuracy: 0.6227 - val_loss: 1.3218 - val_accuracy: 0.6964
Epoch 3/10
60000/60000 [==============================] - 7s 114us/sample - loss: 1.0927 - accuracy: 0.7306 - val_loss: 0.8918 - val_accuracy: 0.7821
Epoch 4/10
60000/60000 [==============================] - 7s 115us/sample - loss: 0.8000 - accuracy: 0.7937 - val_loss: 0.6962 - val_accuracy: 0.8167
Epoch 5/10
60000/60000 [==============================] - 7s 114us/sample - loss: 0.6501 - accuracy: 0.8275 - val_loss: 0.5814 - val_accuracy: 0.8445
Epoch 6/10
60000/60000 [==============================] - 7s 115us/sample - loss: 0.5582 - accuracy: 0.8497 - val_loss: 0.5073 - val_accuracy: 0.8607
Epoch 7/10
60000/60000 [==========================

In [0]:
from tensorflow.keras.optimizers import Adam

In [0]:
model = create_model("relu")
opt = Adam(learning_rate = 0.001, amsgrad=False)
model.compile(optimizer=opt,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

exp_dir = os.path.join(log_dir, "opt2")

tb_callback = tf.keras.callbacks.TensorBoard(log_dir=exp_dir)

model.fit(x_train,
          y_train,
          epochs=10, 
          validation_data=(x_test, y_test), 
          callbacks=[tb_callback])

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 8s 129us/sample - loss: 0.4018 - accuracy: 0.8893 - val_loss: 0.1977 - val_accuracy: 0.9436
Epoch 2/10
60000/60000 [==============================] - 8s 138us/sample - loss: 0.1696 - accuracy: 0.9512 - val_loss: 0.1510 - val_accuracy: 0.9551
Epoch 3/10
60000/60000 [==============================] - 8s 128us/sample - loss: 0.1309 - accuracy: 0.9617 - val_loss: 0.1366 - val_accuracy: 0.9575
Epoch 4/10
60000/60000 [==============================] - 7s 122us/sample - loss: 0.1087 - accuracy: 0.9678 - val_loss: 0.1189 - val_accuracy: 0.9644
Epoch 5/10
60000/60000 [==============================] - 7s 121us/sample - loss: 0.0951 - accuracy: 0.9713 - val_loss: 0.1052 - val_accuracy: 0.9690
Epoch 6/10
60000/60000 [==============================] - 7s 124us/sample - loss: 0.0833 - accuracy: 0.9737 - val_loss: 0.1079 - val_accuracy: 0.9692
Epoch 7/10
60000/60000 [==========================

In [0]:
%tensorboard --logdir "$log_dir"

From the tensor board we can see that both train and validation of momentum optimizer have lower acurracy and higher loss than those of Adam, indicating that in this case Adam has better performance.

### For different initialize strategy :
random normal, or glorot uniform

In [0]:
date = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
log_dir = os.path.join("./tensorboard-logs/", date)
print("Writing logs to", log_dir)

Writing logs to ./tensorboard-logs/20191029-010949


In [0]:
###
def create_model(initializer):
  model = tf.keras.models.Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(128, activation='relu', kernel_initializer=initializer),
    Dense(64, activation='relu'),
    Dense(10, activation='softmax'),
  ])
  return model

In [0]:
model = create_model("random_normal")
opt = SGD(learning_rate=0.001, momentum=0.0, nesterov=False)
model.compile(optimizer=opt,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

exp_dir = os.path.join(log_dir, "random_normal")

tb_callback = tf.keras.callbacks.TensorBoard(log_dir=exp_dir)

model.fit(x_train,
          y_train,
          epochs=10, 
          validation_data=(x_test, y_test), 
          callbacks=[tb_callback])

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 7s 125us/sample - loss: 3.8152 - accuracy: 0.1874 - val_loss: 2.0446 - val_accuracy: 0.4781
Epoch 2/10
60000/60000 [==============================] - 7s 114us/sample - loss: 1.5779 - accuracy: 0.6202 - val_loss: 1.1930 - val_accuracy: 0.7022
Epoch 3/10
60000/60000 [==============================] - 7s 115us/sample - loss: 1.0114 - accuracy: 0.7283 - val_loss: 0.8344 - val_accuracy: 0.7713
Epoch 4/10
60000/60000 [==============================] - 7s 115us/sample - loss: 0.7568 - accuracy: 0.7908 - val_loss: 0.6596 - val_accuracy: 0.8184
Epoch 5/10
60000/60000 [==============================] - 7s 114us/sample - loss: 0.6226 - accuracy: 0.8281 - val_loss: 0.5588 - val_accuracy: 0.8461
Epoch 6/10
60000/60000 [==============================] - 7s 117us/sample - loss: 0.5406 - accuracy: 0.8517 - val_loss: 0.4930 - val_accuracy: 0.8635
Epoch 7/10
60000/60000 [==========================

In [0]:
model = create_model("glorot_uniform")
opt = SGD(learning_rate=0.001, momentum=0.0, nesterov=False)
model.compile(optimizer=opt,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

exp_dir = os.path.join(log_dir, "glorot_uniform")

tb_callback = tf.keras.callbacks.TensorBoard(log_dir=exp_dir)

model.fit(x_train,
          y_train,
          epochs=10, 
          validation_data=(x_test, y_test), 
          callbacks=[tb_callback])

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 7s 123us/sample - loss: 3.4382 - accuracy: 0.2110 - val_loss: 2.0984 - val_accuracy: 0.4516
Epoch 2/10
60000/60000 [==============================] - 7s 114us/sample - loss: 1.7358 - accuracy: 0.5924 - val_loss: 1.3562 - val_accuracy: 0.6850
Epoch 3/10
60000/60000 [==============================] - 7s 115us/sample - loss: 1.1141 - accuracy: 0.7187 - val_loss: 0.8923 - val_accuracy: 0.7741
Epoch 4/10
60000/60000 [==============================] - 7s 114us/sample - loss: 0.7996 - accuracy: 0.7928 - val_loss: 0.6796 - val_accuracy: 0.8149
Epoch 5/10
60000/60000 [==============================] - 7s 113us/sample - loss: 0.6439 - accuracy: 0.8287 - val_loss: 0.5649 - val_accuracy: 0.8460
Epoch 6/10
60000/60000 [==============================] - 7s 114us/sample - loss: 0.5535 - accuracy: 0.8492 - val_loss: 0.4950 - val_accuracy: 0.8629
Epoch 7/10
60000/60000 [==========================

In [0]:
%tensorboard --logdir "$log_dir"

## 3b (optional)
Demonstrate the vanishing gradient problem.

Implement an especially deep neural network and train it on a simple dataset like MNIST. Choose activation functions, initialization strategies, and an optimizer that are likely to cause this behavior. Produce histograms of activations and gradients at various layers during training. What do you see? Next, adjust the parameters above to correct this behavior. Visualize and compare the results.

In [0]:
def create_model():
  model = tf.keras.models.Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(128, activation='signoid'),
    Dense(64, activation='signoid'),
    Dense(64, activation='signoid'),
    Dense(64, activation='signoid'),
    Dense(64, activation='signoid'),
    Dense(64, activation='signoid'),
    Dense(10, activation='softmax'),
  ])
  return model

In [0]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.SGD()

train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')
train_loss = tf.keras.metrics.Mean(name='train_loss')
test_loss = tf.keras.metrics.Mean(name='test_loss')

In [0]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    predictions = model(images)
    loss = loss_fn(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

@tf.function
def test_step(images, labels):
  predictions = model(images)
  t_loss = loss_fn(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [0]:
date = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
log_dir = os.path.join("./tensorboard-logs/", date)
print("Writing logs to", log_dir)

train_writer = tf.summary.create_file_writer(os.path.join(log_dir, "train"))
test_writer = tf.summary.create_file_writer(os.path.join(log_dir, "test"))

Writing logs to ./tensorboard-logs/20191029-015105


In [0]:
EPOCHS = 5

for epoch in range(EPOCHS):
  
  for images, labels in train_ds:
    train_step(images, labels)
    
  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
  print(template.format(epoch+1,
                        train_loss.result(),
                        train_accuracy.result()*100,
                        test_loss.result(),
                        test_accuracy.result()*100))
  
  with train_writer.as_default():
    tf.summary.scalar('accuracy', train_accuracy.result(), step=epoch)
    tf.summary.scalar('loss', train_loss.result(), step=epoch)

    # ====
    # Demo: show how to use histogram summaries
    # Create and log some random data
    # Useful if you're attemping the extra credit question
    # ====
    data = tf.random.normal((32, 100))
    tf.summary.histogram('random', 
                         data,
                         step=epoch, 
                         description='Your description')
    
  with test_writer.as_default():
    tf.summary.scalar('accuracy', test_accuracy.result(), step=epoch)
    tf.summary.scalar('loss', test_loss.result(), step=epoch)
    
  # Reset the metrics for the next epoch
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

Epoch 1, Loss: 0.25563400983810425, Accuracy: 92.61261749267578, Test Loss: 0.2339257150888443, Test Accuracy: 93.14857482910156
Epoch 2, Loss: 0.18101826310157776, Accuracy: 94.74333190917969, Test Loss: 0.1860692948102951, Test Accuracy: 94.51000213623047
Epoch 3, Loss: 0.1717109978199005, Accuracy: 94.97333526611328, Test Loss: 0.1680724322795868, Test Accuracy: 95.01000213623047
Epoch 4, Loss: 0.1624779999256134, Accuracy: 95.25499725341797, Test Loss: 0.16029612720012665, Test Accuracy: 95.13999938964844
Epoch 5, Loss: 0.15450261533260345, Accuracy: 95.5, Test Loss: 0.15948522090911865, Test Accuracy: 95.3499984741211
